# Interaction Location
Function to identify the spatial location of each interaction in a ```.mymridon``` experiment file. This uses the kdtree method defined in [the spatial fidelity notebook](04_spatial_fidelity.ipynb) to create a grid over the experimental space. Then, the (X,Y) coordinate of one of the ant's position at each interaction is extracted and the zone of this interaction in the grid is computed.

In [ ]:
import py_fort_myrmidon as fm
import numpy as np  # Fundamental math library in python.
from datetime import datetime, timedelta  # For convenient handling of time and date
import pandas as pd  # To create a pandas dataframe, an equivalent to an R dataframe
from scipy.spatial import KDTree

### Defining the grid for the spatial fidelity

A spatial tesselation of hexagons is used to define the sites where the observations of the individuals are counted. A nearest neighbour tree (KDTree) is used with the centers of the hexgons mentioned. For a given set of coordinates, a KDTree can be used to efficiently find the seed that is closest to the input coordinates, here the waypoints of the trajectory. The hexagonal boundaries are not defined explicitly. Simply by placing the hexagonal centers accordingly (see ```method ``` variable), the resulting pattern will be hexagonal or squared. The code in the following cell computes the hexagon centers that seed the KDTree.  
This part should probably only be edited w.r.t. the method and the internal radius.

In [ ]:
def define_kdtree(method, xlim, ylim, radius_kdtree):
    if method == "vertical":
        dx = 2 * radius_kdtree
        dy = np.sqrt(3) * radius_kdtree
    elif method == "horizontal":
        dx = np.sqrt(3) * radius_kdtree
        dy = 2 * radius_kdtree
    elif method == "square":
        dx = 2 * radius_kdtree
        dy = 2 * radius_kdtree

    n_x = np.diff(xlim) // dx + 1
    n_y = np.diff(ylim) // dy + 1
    osx = n_x * dx - np.diff(xlim)
    osy = n_y * dy - np.diff(ylim)
    x = np.arange(start=xlim[0] - osx / 2, step=dx, stop=n_x * dx + osx / 2)
    y = np.arange(start=ylim[0] - osy / 2, step=dy, stop=n_y * dy + osy / 2)
    xm, ym = np.meshgrid(x, y)
    if method == "vertical":
        xm[::2] = xm[::2] + radius_kdtree
    elif method == "horizontal":
        ym[:, 1::2] = ym[:, 1::2] + radius_kdtree

    sites = np.stack([xm.flatten(), ym.flatten()], axis=1)
    kdtree = KDTree(sites)
    return kdtree, sites

### Function to query the kdtree for the location of specific interactions

In [ ]:
def interaction_loc_query(
    myrmidon_exp, t_start, t_end, method_kdtree, xlim, ylim, radius_kdtree
):
    """
    Function to query a kdtree to calculate zone of where an interaction happened
    :param myrmidon_exp: Myrmidon experiment (not path)
    :param t_start: Start of experiment duration in local time
    :param t_end: End of experiment duration in local time
    :param method_kdtree: Method to be used for kdtree - vertical, horizontal or square (first 2 are hexagonal shapes)
    :param xlim: The maximum value in the x-axis, corresponding to the image size
    :param ylim: The maximum value in the y-axis corresponding to the image size
    :param radius_kdtree: The radius to be used for the creation of the kdtree
    :return: Returns the KDTree used for the query and a dataframe containing X,Y coordinates of ant 1, IDs of ant 1 and 2 involved in the interaction, time of interaction and the zone in the KDTree that the interaction occured in.
    """
    kdtree = define_kdtree(method_kdtree, xlim, ylim, radius_kdtree)
    m = fm.Matcher.InteractionType(1, 1)
    # Get all interactions within a specific time window
    _, interactions = fm.Query.ComputeAntInteractions(
        myrmidon_exp, start=t_start, end=t_end, matcher=m
    )
    # Create interactions dataframe
    interaction_df = pd.DataFrame(
        [x.Trajectories[0].Trajectory.Positions[0, 1] for x in interactions],
        columns=["x"],
    )  # X coordinatees of the first ant first point in trajectory related to interaction
    interaction_df["y"] = [
        x.Trajectories[0].Trajectory.Positions[0, 2] for x in interactions
    ]  # Y coordinates of first ant first point in trajectory related to interaction
    interaction_df["antID1"] = [
        x.Trajectories[0].Trajectory.Ant for x in interactions
    ]  # ID of first ant
    interaction_df["antID2"] = [
        x.Trajectories[1].Trajectory.Ant for x in interactions
    ]  # ID of second ant
    interaction_df["Time"] = [
        x.Trajectories[0].Trajectory.Start for x in interactions
    ]  # Time of interaction
    # Calculate zone of each X,Y coordinate using kdtree query
    dist, zone_indices = kdtree.query(interaction_df.iloc[:, 0:2])
    interaction_df["zone"] = zone_indices
    return kdtree, interaction_df